In [4]:
class demander:
    def __init__(self, location, objective_function, domain):
        self.location = location
        self.objective_function = objective_function
        self.domain = domain

    def location_get(self):
        return self.location

In [3]:
ali = demander(1, 2, 3)
print(ali.location)

1
